# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data = pd.read_csv('output_data/cities.csv')
data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.1203,-134.9692,74.01,71,0,14.43,PF,1632280834
1,Carutapera,-1.1950,-46.0200,78.10,83,24,7.65,BR,1632280834
2,Busselton,-33.6500,115.3333,78.04,55,0,4.47,AU,1632280834
3,Nouadhibou,20.9310,-17.0347,71.58,83,0,16.11,MR,1632280835
4,Banda Aceh,5.5577,95.3222,83.97,65,100,13.18,ID,1632280835


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
location = data[['Lat', 'Lng']]
fig = gmaps.figure(center =(30,5),zoom_level =2)
#add heatmap layer
heat_layer = gmaps.heatmap_layer(location, weights = data['Humidity'], dissipating=False, max_intensity=100, point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
good_city_df = data.loc[((data['Max Temp']<80) & 
                         (data['Max Temp']>70) & 
                         (data['Wind Speed']<10) & 
                         (data['Cloudiness'] == 0))]
good_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,Busselton,-33.6500,115.3333,78.04,55,0,4.47,AU,1632280834
29,Hobart,-42.8794,147.3294,73.11,41,0,5.01,AU,1632280844
129,Gladstone,-23.8500,151.2500,73.33,23,0,8.05,AU,1632280752
149,Esperance,-33.8667,121.9000,71.56,38,0,9.57,AU,1632280889
169,Aksu,41.1231,80.2644,75.00,23,0,4.94,CN,1632280896
264,Kiryat Gat,31.6100,34.7642,76.41,76,0,3.56,IL,1632280740
305,San Quintín,30.4833,-115.9500,72.77,75,0,8.99,MX,1632280946
339,Cascais,38.6979,-9.4215,70.47,44,0,8.01,PT,1632280958
347,Kashan,33.9831,51.4364,75.69,18,0,8.93,IR,1632280961
413,Hangu,39.2489,117.7892,78.84,56,0,9.91,CN,1632280986


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = good_city_df.reset_index(drop=True)
hotel_df['Hotel Name'] =''
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Busselton,-33.6500,115.3333,78.04,55,0,4.47,AU,1632280834,
1,Hobart,-42.8794,147.3294,73.11,41,0,5.01,AU,1632280844,
2,Gladstone,-23.8500,151.2500,73.33,23,0,8.05,AU,1632280752,
3,Esperance,-33.8667,121.9000,71.56,38,0,9.57,AU,1632280889,
4,Aksu,41.1231,80.2644,75.00,23,0,4.94,CN,1632280896,
5,Kiryat Gat,31.6100,34.7642,76.41,76,0,3.56,IL,1632280740,
6,San Quintín,30.4833,-115.9500,72.77,75,0,8.99,MX,1632280946,
7,Cascais,38.6979,-9.4215,70.47,44,0,8.01,PT,1632280958,
8,Kashan,33.9831,51.4364,75.69,18,0,8.93,IR,1632280961,
9,Hangu,39.2489,117.7892,78.84,56,0,9.91,CN,1632280986,


In [6]:
for index,row in hotel_df.iterrows():
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    city = row['City']
    lat = row['Lat']
    lng = row['Lng']
    
    #set location
    location = f'{lat},{lng}'
    
    #set params
    params = {'location':location,'keyword':'hotel','radius':5000,'type':'lodging','key':g_key}
    
    #make API call
    response = requests.get(base_url,params = params).json()
    #pprint(response)
    try:
        hotel_df.loc[index,'Hotel Name'] = response['results'][0]['name']
    except(KeyError,IndexError):
        print('missing value. Skipped')
hotel_df    

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Busselton,-33.6500,115.3333,78.04,55,0,4.47,AU,1632280834,Esplanade Hotel
1,Hobart,-42.8794,147.3294,73.11,41,0,5.01,AU,1632280844,Islington Hotel
2,Gladstone,-23.8500,151.2500,73.33,23,0,8.05,AU,1632280752,Rydges Gladstone
3,Esperance,-33.8667,121.9000,71.56,38,0,9.57,AU,1632280889,Esperance Chalet Village
4,Aksu,41.1231,80.2644,75.00,23,0,4.94,CN,1632280896,Pudong Holiday Hotel
5,Kiryat Gat,31.6100,34.7642,76.41,76,0,3.56,IL,1632280740,Desert Gat
6,San Quintín,30.4833,-115.9500,72.77,75,0,8.99,MX,1632280946,Don Eddie's Landing
7,Cascais,38.6979,-9.4215,70.47,44,0,8.01,PT,1632280958,Hotel Londres Estoril
8,Kashan,33.9831,51.4364,75.69,18,0,8.93,IR,1632280961,هتل
9,Hangu,39.2489,117.7892,78.84,56,0,9.91,CN,1632280986,Hanting Hotel


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))